<a href="https://colab.research.google.com/github/sp589/community_detection/blob/main/leader_based_community_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# community detection

In [1]:
# import all dependency
import time as time
import numpy as np
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
import networkx as nx
import math as math
import pandas as pd
import matplotlib.pyplot as plt
from random import randint
import zipfile

In [2]:
# analyissis essential tool
def nodelist_to_analysis(node_list,community):
    F_node_list=node_list.copy()
    for n in range(len(community)):
        for m in community[n]:
            F_node_list[node_list.index(m)]=n
    return F_node_list

def nodelist_set(community):
    c_set=[]
    for i in range((len(community))):
        c_set.append(set(community[i]))
    return c_set

In [3]:
def intersection(c, x): 
    return list(set(c) & set(x))

def Union(c, x): 
    final_list = list(set(c) | set(x))  
    return final_list

In [4]:
# similarity measure
def similarity_measure(leader,i,Graph,method):
    if method=="HDI":                                 # HDI INDEX METHOD
        kx=Graph.degree(leader)
        c=[n for n in Graph.neighbors(leader)]
        x=[n for n in Graph.neighbors(i)]               
        ky=Graph.degree(i)
        lenght_of_common_nodes=len(intersection(c, x)) 
        similarity = lenght_of_common_nodes/(max(kx,ky))
        #return similarity
    elif method=="HPI":                                # HPI INDEX METHOD
        kx=Graph.degree(leader)
        c=[n for n in Graph.neighbors(leader)]
        x=[n for n in Graph.neighbors(i)]               #  
        ky=Graph.degree(i)
        lenght_of_common_nodes=len(intersection(c, x)) 
        similarity = lenght_of_common_nodes/(min(kx,ky))
        #return similarity
    elif method=="JACCARD":                           # JACCARD INDEX METHOD
        kx=Graph.degree(leader)
        c=[n for n in Graph.neighbors(leader)]
        x=[n for n in Graph.neighbors(i)]              
        lenght_of_common_nodes=len(intersection(c, x)) 
        lenght_of_union=len(Union(c, x))
        if (lenght_of_union>0):
            similarity =  lenght_of_common_nodes/ lenght_of_union
        else:
            similarity=0
        #return similarity 
    elif method=="myfollower":
        kx=Graph.degree(leader)                        # SALTON INDEX METHOD
        c=[n for n in Graph.neighbors(leader)]
        x=[n for n in Graph.neighbors(i)] 
        ky=Graph.degree(i)
        #SQ=math.sqrt(kx*ky)
        lenght_of_common_nodes=len(intersection(c, x)) 
        similarity = lenght_of_common_nodes/ky
        
    else:
        kx=Graph.degree(leader)                        # SALTON INDEX METHOD
        c=[n for n in Graph.neighbors(leader)]
        x=[n for n in Graph.neighbors(i)] 
        ky=Graph.degree(i)
        SQ=math.sqrt(kx*ky)
        lenght_of_common_nodes=len(intersection(c, x)) 
        similarity = lenght_of_common_nodes/SQ
    return round(similarity,4)

In [5]:
# plot graph with community
def community_show(Graph,community):
    color = []
    for i in range((len(community))):
        color.append('#%06X' % randint(0, 0xFFFFFF)) 
    nodelist=[n for n in Graph]
    color_map=nodelist.copy()
    for i in range((len(community))):
        for node in Graph:
            if node in community[i]:
                color_map[nodelist.index(node)]=color[i]
    #nx.draw(Graph, node_color=color_map,with_labels=True)
    pos = nx.spring_layout(Graph)
    import warnings
    warnings.filterwarnings('ignore')
    plt.style.use('fivethirtyeight')
    plt.rcParams['figure.figsize'] = (20, 15)
    plt.axis('off')
    nx.draw_networkx(Graph,pos, with_labels = True,node_color=color_map, node_size = 35)
    plt.show()
    #plt.title("Graph")
    #plt.savefig((easygui.filesavebox())+"."+"png")
    #plt.show()

In [6]:
def leader_selection(Graph,method):
    if method=="EIGEN":
        centrality=nx.eigenvector_centrality(Graph,max_iter=100000000)
        c_valu_l=list(centrality.values())
        c_node=list(centrality.keys())
    elif method=="BETWEEN":
        centrality=nx.betweenness_centrality(Graph)
        c_valu_l=list(centrality.values())
        c_node=list(centrality.keys())
    else:
        centrality=nx.degree_centrality(Graph)
        c_valu_l=list(centrality.values())
        c_node=list(centrality.keys())
    return c_node[c_valu_l.index(max(c_valu_l))]

In [7]:
def check_for_node_left(temp_c,temp_r,Graph):
    H=Graph.subgraph(temp_r)
    connected_components_list=list(nx.connected_components(H))
    for i in connected_components_list:
        if (len(i))==1:
            temp_c.append((list(i))[0])
            #temp_r.remove((list(i))[0])
    for node in temp_c:
        Graph.remove_node(node)
    return temp_c,Graph

In [8]:
def Leader_Based(Graph,thershold,sim_method,leader_sel):
    community=[]
    temp_c=[]
    temp_r=[]
    while(len(Graph)!=0):
        leader=leader_selection(Graph,leader_sel)
        for i in Graph:
            similarity=similarity_measure(leader,i,Graph,sim_method)
            #print(similarity)
            if similarity > thershold: 
                temp_c.append(i)
            else:
                temp_r.append(i)
        com,Graph=check_for_node_left(temp_c,temp_r,Graph)
        community.append(com.copy())
        temp_c.clear()
        temp_r.clear()
    #print("lbcd",len(community))
    return community

In [9]:
def real_network(name):
    if name=="football":
        zf=zipfile.ZipFile("dataset/football.zip")
        txt=zf.read('football.txt').decode()
        gml=zf.read('football.gml').decode()
        gml=gml.split('\n')[1:]
        G=nx.parse_gml(gml)
    elif name=="dolphins":
        zf=zipfile.ZipFile("dataset/dolphins.zip")
        txt=zf.read('dolphins.txt').decode()
        gml=zf.read('dolphins.gml').decode()
        gml=gml.split('\n')[1:]
        G=nx.parse_gml(gml)
    elif name=="youtube":
        G= nx.read_edgelist('dataset/youtube-combined.txt', create_using = nx.Graph(), nodetype = int)
    else:
        G=nx.karate_club_graph()
    return G

start=time.process_time_ns()
#Graph_original=generate_graph(400)
Graph_original=real_network("youtube")
#nx.draw(,with_labels=False)
pos = nx.spring_layout(Graph_original)
import warnings
warnings.filterwarnings('ignore')
plt.style.use('fivethirtyeight')
plt.rcParams['figure.figsize'] = (20, 15)
plt.axis('off')
nx.draw_networkx(Graph_original,pos, with_labels = False, node_size = 35)
plt.show()
execution=(time.process_time_ns()-start)/(1000000000)
print("execution time in sec",execution)
#plt.show()

#Graph_original=nx.karate_club_graph()
Graph_original=generate_graph(400)
Graph=Graph_original.copy()
nx.draw(Graph,with_labels=True)
plt.show

In [10]:
from sklearn import metrics
from sklearn.metrics.cluster import normalized_mutual_info_score
def community_anlysis(GRAPH,node_list,GT,COMM):
    n1=nodelist_to_analysis(node_list,GT)
    N1=nodelist_to_analysis(node_list,COMM)
    COMM_SET=nodelist_set(COMM)
    nmi=normalized_mutual_info_score(n1,N1)
    ARI=metrics.adjusted_rand_score(n1,N1)
    modularity=nx.algorithms.community.modularity(GRAPH,COMM_SET)
    #print("NMI: ",modularity)
    return ARI,nmi,modularity
    


In [11]:

def synthetic_ground_truth(GT):
    community_list=[]
    community_set=[]    
    communities=list(GT)
    community_node=[]
    for i in range(len(communities)):
        community_list.append(list(communities[i])) 
    for i in range(len(community_list)):
        for j in community_list[i]:
            community_node.append(j)
    return community_node,community_list

In [12]:
# synthetic Network
from networkx.generators.community import LFR_benchmark_graph as lfr

#def generate_graph(n_low, n_high):
def generate_graph(n,mu):
    #n = np.random.randint(n_low, n_high)
    tau1 = 3
    tau2 = 1.5
    #tau2=1.1
    max_iters=100
    #mu = np.random.uniform(0.03, 0.5)
    #mu=0.5
    max_degree = int(0.1 * n)
    max_community = int(0.1 * n)
    #average_degree = 10
    if n<=1000:
      average_degree = 8
    elif n>1000 and n<=1600:
      average_degree = 18
    elif n>1600 and n<=2500:
      average_degree = 20
    elif n>2500 and n<=3000:
      average_degree = 28
    elif n>3000 and n<=4000:
      average_degree= 28
    elif n>4000 and n<=4500:
      average_degree = 30
    elif n==4800:
      average_degree = 42
    elif n>1000 and n<=5000 and n!=4800:
      average_degree = 36
    else:
      average_degree = 32
    G = lfr(
        n,
        tau1,
        tau2,
        mu,
        average_degree=average_degree,
        max_community=max_community,
        max_degree=max_degree,
        max_iters=10 * n * max_iters,
        seed=10
    )
    return G

In [ ]:
# leader based community detection
import numpy as np
thershold_range=np.linspace(0.0,0.9,1000,0.001)
#thershold_range=[0.073,0.074,0.075,0.076,0.077,0.078,0.079]
community_have=[]
detected_community=[]
node=[]
leader=[]
siml=[]
execution_time=[]
PAR_ARI=[]
PAR_NMI=[]
PAR_mod=[]
density=[]
simelarity_method=["myfollower"] # "JACCARD" ,"SALTON" ,"HDI", "HPI"
leader_sel=["EIGEN","DEGREE"]  # "EIGEN","PAGE","DEGREE"
mu=0.3
n=2500
n1=n
i=0
j=0
kt=[]
node_range=3000
while(n<node_range):
    Graph_original=generate_graph(n,mu)
    communities = {frozenset(Graph_original.nodes[v]["community"]) for v in Graph_original}
    node_list,GT=synthetic_ground_truth(communities)
    print('start for node ',n)
    for i in range(len(simelarity_method)):
        for j in range(len(leader_sel)):
            #start=time.process_time_ns()
            for thershold in thershold_range:
                Graph=Graph_original.copy()
                kr=round(thershold,4)
                
                #kr=thershold
                start=time.process_time_ns()
                community1=Leader_Based(Graph,kr,simelarity_method[i],leader_sel[j])
                #print('k',kr,'l',len(community1))
                kt.append(kr)
                execution=(time.process_time_ns()-start)/(1000000000)
                #print(" done kr:",kr,"no community",len(community),end="")
                ARI,NMI,mod=community_anlysis(Graph_original,node_list,GT,community1)
                PAR_ARI.append(round(ARI,6))
                PAR_NMI.append(round(NMI,6))
                PAR_mod.append(round(mod,6))
                community_have.append(len(GT))
                node.append(n)
                density.append(nx.density(Graph_original))
                detected_community.append(len(community1))
                execution_time.append(execution)
                leader.append(leader_sel[j])
                siml.append(simelarity_method[i])
                if (len(community1))>=(len(GT)):
                    break
            #print("s",simelarity_method[i],"l",leader_sel[j])
    
    print("done for node")
    n=n+100
DataFrame=pd.DataFrame({'node':node,'density':density,'K':kt,'leader':leader,'similarity_method':siml,'No Community':community_have ,'Detected Community':detected_community,"MODULARITY":PAR_mod,"NMI":PAR_NMI,"ARI":PAR_ARI,"time":execution_time}).to_csv('leader_based_community_'+str(n1)+'_'+str(node_range)+'_'+str(mu)+'.csv')
print("finally done")
    #rint("no community",len(community))
    #rint("execution time in sec",execution)

start for node  2500
done for node
start for node  2600
done for node
start for node  2700
done for node
start for node  2800


In [ ]:
# leader based community detection
thershold=0.3
community_have=[]
detected_community=[]
node=[]
execution_time=[]
mu=0.3
n=2500
node_range=2600
while(n<node_range):
    Graph_original=generate_graph(n,mu)
    print("density",nx.density(Graph_original))
    communities = {frozenset(Graph_original.nodes[v]["community"]) for v in Graph_original}
    node_list,GT=synthetic_ground_truth(communities)
    #print(len(list(communities)))
    simelarity_method="HDI" # "JACCARD" ,"SALTON" ,"HDI", "HPI"
    leader_sel="EIGEN "  # "EIGEN","PAGE","DEGREE"
    Graph=Graph_original.copy()
    start=time.process_time_ns()
    community1=Leader_Based(Graph_original,thershold,simelarity_method,leader_sel)
    community_anlysis(Graph,node_list,GT,community1)
    execution=(time.process_time_ns()-start)/(1000000000)
    print("done for no of node:",n)
    node.append(n)
    community_have.append(len(GT))
    detected_community.append(len(community1))
    execution_time.append(execution)
    n=n+100
    
    print("no community",len(community1))
    print("execution time in sec")